In [2]:
using MagneticLaplacianSparsifier
using Graphs, MetaGraphs, Random, IterTools, LinearAlgebra, GraphPlot, Arpack, Plots, SparseArrays
using Statistics
using Measures
using StatsBase

using CairoMakie
CairoMakie.activate!(type="png")
set_theme!(resolution=(400, 400))
CairoMakie.inline!(true)

using GraphMakie: graphplot
using GraphMakie: graphplot!

using CairoMakie.Colors
using GeometryBasics: Point

In [5]:
spanning_graph = "CRSF"

##  default parameters
# number of nodes on the side
n_side = 3

# interval [a,b]^2
a = 0
b = 1

# for sampling
absorbing_node = false
ust = false # for spanning trees

# proba to have an outlier edge
η = 0.
# uniform noise on edges
noise = 0.
# regularization parameter
q = 0.

if spanning_graph == "ST"
    absorbing_node = true
    ust = true
    η = 0.
    noise = 0.
elseif spanning_graph == "SF"
    q = 0.1
elseif spanning_graph == "CRSF"
    η = 0.2
    noise =0.1
elseif spanning_graph == "MTSF"
    q = 0.05
    η = 0.2
    noise = 0.
end

# edges weights?
weighted = false

# random generator
rng = Random.default_rng()

# number of nodes
n_v = n_side^2

# grid spacing
grid_spacing = (b-a)/(n_side - 1) 
# grid coordinates
x = flat_square_2d_grid(n_v, a, b)
# for plotting
function mylayout(g::AbstractGraph)
    # create iterator going over x-coord and y-coord 
    return Point.(zip(x[:, 1], x[:, 2]))
end

# planted ranking score
planted_score = randperm(rng,n_v);

meta_g = MetaGraph(n_v)

for u in 1:n_v
    for v in 1:n_v
        if u < v
            edge_exits = (norm(x[u,:]-x[v,:],1) < grid_spacing + 1e-10)
            if edge_exits 
                h_u = planted_score[u]
                h_v = planted_score[v]
                θ = (h_u - h_v) * π / (n_v - 1)
                if (rand(rng) < η)
                    θ = rand(rng, (-n_v + 1):(n_v - 1)) * π / (n_v - 1)
                else
                    θ *= 1.0 + noise * 2 * (rand(rng) - 0.5)
                end
                add_edge!(meta_g, u, v, :angle, θ)
            end
        end
    end
end



In [35]:
g = meta_g

mtsf = MetaGraph(nv(g))

nv_mtsf = 0
weight = 1.0
roots = []
nodes_in_cycles = Vector[]
reverse_order_branches = Vector[]

list_of_graphs = []
temp_g = MetaGraph(nv(g))

# Initialize the random walk
walk = []
unvisited = Set(vertices(g))

#fix a root if necessary
if absorbing_node
    ab_node = rand(rng, unvisited)
    push!(roots, ab_node)
    setdiff!(unvisited, ab_node)
    nv_mtsf += 1
end

# Start the random walk
n0 = rand(rng, unvisited)
# add n0 to walk
push!(walk, n0)
# mark n0 as visited
setdiff!(unvisited, n0)

while nv_mtsf < nv(g)

    # check if n0 is a root
    n0_is_root = false
    if q > 1e-10 # namely if q large enough
        #n0_is_root = rand(rng) < q / (q + degree(g, n0))
        n0_is_root = step_to_root(rng, g, n0, q, weighted)
    end

    if n0_is_root # if n0 is indeed a root
        # record the rooted branch in mtsf
        push!(roots, n0)
        add_edges_from!(mtsf, consecutive_pairs(walk))
        nv_mtsf += length(walk)
        # mark branch as visisted
        setdiff!(unvisited, walk)

        # record branch w/o  root
        push!(reverse_order_branches, walk[1:(end - 1)])
        # restart with a new n0 uniformly among the unvisited nodes
        n0 = restart_walk_from_unvisited_node!(rng, walk, unvisited)
        continue
    end

    # walk to a neighbour wrt degree distribution
    n1 = rand_step(rng, g, n0, weighted)
    # add neighbour to walk
    push!(walk, n1)

    if n1 in unvisited
        setdiff!(unvisited, n1)
        n0 = n1  # if n1 unvisited continue the walk
    
    # otherwise n1 is already visited    
    elseif (degree(mtsf, n1) > 0 || n1 in roots) || (n1 in roots && ust)
        print("branch merged ")
        add_edges_from!(mtsf, consecutive_pairs(walk))
        nv_mtsf += length(walk) - 1
        setdiff!(unvisited, walk)

        push!(reverse_order_branches, walk[1:(end - 1)])

        # store mtsf
        print(mtsf)
        push!(list_of_graphs,mtsf)

        n0 = restart_walk_from_unvisited_node!(rng, walk, unvisited)
        print("and restart walk \n")

    else  # if n1 in walk: identify unique cycle/loop in walk with knot n1

        print("cycle formed: ")
        idx_n1 = findfirst(x -> x == n1, walk)
        cycle_nodes = @view walk[idx_n1:end]
        keep = false # by default pop cycle
        alpha = 0
        if !ust # if we are not in the spanning tree case, we can try to accept the cycle
            keep, alpha = keep_cycle(rng, g, consecutive_pairs(cycle_nodes))
        end

        if keep # keep cycle
            print(" accepted ")
            weight *= max(alpha, 1)
            add_edges_from!(mtsf, consecutive_pairs(walk))
            nv_mtsf += length(walk) - 1 # since walk contains twice the knot
            setdiff!(unvisited, walk)

            # record cycle nodes: remove starting node so that it appears only once
            push!(nodes_in_cycles, cycle_nodes[2:end])
            # record branch without the knot
            push!(reverse_order_branches, walk[1:(idx_n1 - 1)])

            n0 = restart_walk_from_unvisited_node!(rng, walk, unvisited)
            print("and restart walk\n")

            # store mtsf
            print(mtsf)
            mtsf_temp = mtsf
            push!(list_of_graphs,mtsf_temp)

        else  # pop cycle but keep the knot
            print(" popped ")
            union!(unvisited, cycle_nodes)
            setdiff!(unvisited, n1)  # remove n1 which was part of cycle_nodes
            resize!(walk, idx_n1)
            # restart from the knot
            n0 = n1
            print("and restart walk\n")
        end
    end
end


cycle formed:  popped 

and restart walk
cycle formed:  popped and restart walk
cycle formed:  popped and restart walk
cycle formed:  popped and restart walk
cycle formed:  popped and restart walk
cycle formed:  popped and restart walk
cycle formed:  popped and restart walk
cycle formed:  popped and restart walk
cycle formed:  popped and restart walk
cycle formed:  accepted and restart walk
{9, 6} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)cycle formed:  popped and restart walk
cycle formed:  popped and restart walk
cycle formed:  popped and restart walk
branch merged {9, 8} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)and restart walk 
branch merged {9, 9} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)and restart walk 


In [36]:
list_of_graphs

3-element Vector{Any}:
 {9, 9} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)
 {9, 9} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)
 {9, 9} undirected Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)